# Hyperspy Tutorial

## EELS analysis of perovskite oxides

This tutorial shows the various functionalities in HyperSpy which is used to analyse Electron Energy Loss Spectroscopy data, using EELS datasets from perovskite oxide heterostructure.

It assumes some knowledge on how to use HyperSpy, like loading datasets and how the basic signals work.

######  Requirement

HyperSpy 0.8.5

###### Notebook

This notebook was used for the HyperSpy workshop at the Norwegian University of Science and Technology for the Scandem 2016 conference, 7 June 2016.

###### Data

The data was acquired on a Jeol ARM200cF using a Gatan Quantum ER with DualEELS capabilites.

###### Sample

La0.7Sr0.3MnO3 thin films deposited on SrTiO3. In the Fine Structure example parts of the film has been exposed to a very long electron beam exposure, inducing oxygen vacancies.

###### Author

Magnus Nord

###### Table of contents

1. <a href='#simple_quant'> Simple quantification</a>
2. <a href='#curve_fitting_quant'> Curve fitting quantification</a>
3. <a href='#fine_structure_analysis'> Fine structure analysis</a>
4. <a href='#fine_structure_ok'> Fine structure oxygen-K edge</a>

Firstly we use some Jupyter magic to import the right plotting libraries, then import HyperSpy

In [ ]:
%matplotlib qt4

In [ ]:
import hyperspy.api as hs

## <a id='simple_quant'></a> 1. Simple quantification


First we take a look at an EELS line scan across an La0.7Sr0.3MnO3/SrTiO3 thin film. The core loss data has several peaks: Ti-L23, O-K, Mn-L23 and La-M54. We can navigate the line scan using the navigation window, and by moving the red line.

In [ ]:
s = hs.load("datasets/LSMO_STO_linescan.hdf5")

In [ ]:
s.plot()

Now we can quantifiy the first edge (Ti-L23). Firstly by removing the plasmon background, then cropping the signal to only include the Ti-L23 edge

In [ ]:
s.remove_background()

In [ ]:
s.crop_spectrum()

Then we can sum the energy axis to get the relative Ti-content

In [ ]:
s_ti = s.sum(-1)

In [ ]:
s_ti.plot()

## <a id='curve_fitting_quant'></a> 2. Curve fitting quantification

Now, lets do some more advanced quantification using HyperSpy's extensive modelling framework. Firstly we load the low loss and core loss spectra.

In [ ]:
s_ll = hs.load("datasets/LSMO_STO_linescan_low_loss.hdf5")

In [ ]:
s = hs.load("datasets/LSMO_STO_linescan.hdf5")

Here, the metadata has been populated with some of the experimental parameters:

In [ ]:
s.metadata

Firstly, we make sure the energy scale is properly aligned by using the zero loss peak in the low loss spectrum.

In [ ]:
s_ll.align_zero_loss_peak(subpixel=True, also_align=[s])

We have to add the elements which is present in the sample to `s`

In [ ]:
s.add_elements(('Mn','O','Ti','La'))

Then we make a model out of the core loss spectrum. The low loss spectrum is convolved with the model, which means plural scattering is automatically taken into account. In addition this leads to better fits.

In [ ]:
m = s.create_model(ll=s_ll)

The model new consist of many different EELSCLEdge components, including a component for the plasmon background

In [ ]:
m.components

We can fit the model to the experimental data by using the `multifit` function, with the `smart` fitting. Which is fits in a way optimized for EELS data, by fitting from the lowest to the highest energy losses.

In [ ]:
m.multifit(kind='smart')

In [ ]:
m.plot()

This fitted mostly ok, but it is still not very good. This is due to the fine structure not currently taken into account by the model. To get a good fit, we can either not fit to the fine structure regions, or model them somehow.
The easiest way is defining certain regions as fine structure:

In [ ]:
m.enable_fine_structure()

This will produce a much better fit, but will be much slower (~2 minutes).

In [ ]:
m.multifit(kind='smart')

Now the fit is much better, due to the model taking into account the fine structure.

In [ ]:
m.plot()

Now we can can have a look at the intensity from the individual EELS-edges

In [ ]:
m.components.Ti_L3.intensity.plot()

In [ ]:
m.components.La_M5.intensity.plot()

In [ ]:
m.components.Mn_L3.intensity.plot()

## <a id='fine_structure_analysis'></a> 3. Fine structure analysis

Here we take a look at a linescan from a La0.7Sr0.3MnO3 thin film, where parts of the film has been bombarded with the electron beam for an extended time.

In [ ]:
s = hs.load("datasets/LSMO_linescan.hdf5")

Using the moving the red line in the there is clearly something going on in the middle on both the oxygen and the manganese edges. In addition, there are some thickness changes during the line scan.

In [ ]:
s.plot()

Using the low loss signal, we make sure the energy scale is properly calibrated

In [ ]:
s_ll = hs.load("datasets/LSMO_linescan_low_loss.hdf5")

In [ ]:
s_ll.plot()

The zero loss peak is not well aligned at 0 eV energy loss, so we should align it and the core loss

In [ ]:
s_ll.align_zero_loss_peak(subpixel=True, also_align=[s])

Now the zero loss peak has been shifted to 0 energy loss, and likewise the core loss spectrum `s` has also been aligned

In [ ]:
s_ll.plot()

We can also calculate the relative thickness using the low loss. We'll have to specify the end of the zero loss beam, which for cold field emissions guns 3.0 eV seems to work well.

In [ ]:
s_thickness = s_ll.estimate_thickness(threshold=3.0)

This gives the relative thickness and, as expected, there is an increase towards the end of the line scan

In [ ]:
s_thickness.plot()

## <a id='fine_structure_ok'></a> 4. Fine structure: oxygen K-edge
Lets take a closer look at the oxygen-K edge, firstly by removing the plasmon background, then cropping the spectrum to only include the oxygen-K edge. Note: this will overwrite the `s` spectrum with the cropped one. 

In [ ]:
s.remove_background()

In [ ]:
s.crop_spectrum()

This makes it much easier to compare the different positions. Pressing 'e' with the spectrum window highlighted gives a second spectrum picker, which can be moved independently of the first one

We can then do fourier ratio deconvolution to remove the effects of plural scattering

In [ ]:
s_deconvolved = s.fourier_ratio_deconvolution(s_ll)

In [ ]:
s_deconvolved.plot()

### Fine structure modelling

Having had a qualitativily look at the data, we can try to quantify some of these changes. We do this by making making a model of the oxygen-K edge signal. As we've already removed the background, we set `auto_background=False`.

In [ ]:
m = s.create_model(ll=s_ll, auto_background=False)

So currently, the model does not contain any components

In [ ]:
m.components

We can try to model some of the fine structure with Gaussians

In [ ]:
g1 = hs.model.components.Gaussian()

In [ ]:
m.append(g1)

This added the gaussian component to the model

In [ ]:
m.components

Then we can fit this Gaussian to the largest of the O-K peaks by dragging a span over the peak between 528 and 533 eV. Run it first with the "Only Current" option ticked, then run it without to fit the whole dataset

In [ ]:
m.fit_component(g1)

Having fitted the Gaussian to the experimental data, we can plot how the Gaussian three parameters change over the line scan: A, sigma and centre. The A changes quite a bit, which is probably (among others) related to thickness changes. However, there are clear changes in the sigma parameter in the region with the electron beam damage

In [ ]:
g1.plot()

Using the same method we can also fit the second largest peak between 535 and 541 eV

In [ ]:
g2 = hs.model.components.Gaussian()

In [ ]:
m.append(g2)

In [ ]:
m.fit_component(g2)

However, this time the final fit does not look very good. This is due to the two components being fitted independently of eachother. We should fit both of them at the same time. Firstly, we have to set the `signal_range` which is where the model will fit to the experimental data. Here we select the region spanning the two major peaks (528-541 eV)

In [ ]:
m.set_signal_range()

In [ ]:
m.multifit()

After fitting, we reset the signal range so we can see the full range of the signal

In [ ]:
m.reset_signal_range()

In [ ]:
m.plot()

Lastly, we can fit the small "pre-peak" as well. First we "lock" the two Gaussian we have already fitted.

In [ ]:
g1.set_parameters_not_free()

In [ ]:
g2.set_parameters_not_free()

Then we add another Gaussian, and fit it using `fit_component` between 522 and 527 eV

In [ ]:
g3 = hs.model.components.Gaussian()

In [ ]:
m.append(g3)

In [ ]:
m.fit_component(g3)

Then we set the signal range to cover all the three peaks, from 520 eV to 541 eV

In [ ]:
m.set_signal_range()

In [ ]:
m.multifit()

This fits all the three components to the experimental data, which hopefully gives a good fit

In [ ]:
m.reset_signal_range()

In [ ]:
m.plot()

Then we can save the sigma of the different components as signals:

In [ ]:
s_g1s = g1.sigma.as_signal()

In [ ]:
s_g2s = g2.sigma.as_signal()

In [ ]:
s_g3s = g3.sigma.as_signal()